In [32]:
import json
import shutil
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [41]:
df = pd.read_csv('sites/ugalla_issa_chimp_video_data.csv', encoding = "ISO-8859-1")

In [42]:
df.columns

Index(['new_row_id', 'country', 'research_site', 'genus', 'species',
       'cam_coverage_area', 'location_metadata', 'habitat', 'utm_zone',
       'utm_long', 'utm_lat', 'cam_id', 'vid_res', 'vid_duration',
       'panaf_datasheet_comments', 'year', 'month', 'day', 'time_hr',
       'time_min', 'cell_id', 'subfolder', 'video_file_name', 'event_id',
       'min_number_chimps_per_video', 'max_number_chimps_per_video',
       'age_class', 'sex', 'tool_use', 'vocalization', 'bipedal',
       'camera_reaction', 'behavioral_context', 'other_species',
       'additional_comments', 'record_type'],
      dtype='object')

In [43]:
df.subfolder

0       uga_vid29_231560_9396216_20140704
1       uga_vid29_231560_9396216_20140704
2       uga_vid29_231560_9396216_20140704
3       uga_vid29_231560_9396216_20140704
4       uga_vid29_231560_9396216_20140704
                      ...                
1336    uga_vid32_230040_9391979_20140823
1337    uga_vid32_230040_9391979_20140823
1338    uga_vid32_230040_9391979_20140823
1339    uga_vid32_230040_9391979_20140823
1340    uga_vid32_230040_9391979_20140823
Name: subfolder, Length: 1341, dtype: object

In [44]:
df.video_file_name

0           pict0056
1           pict0057
2           pict0057
3           pict0057
4           pict0057
            ...     
1336    PICT0359.JPG
1337    PICT0360.JPG
1338    PICT0361.JPG
1339    PICT0362.JPG
1340    PICT0363.JPG
Name: video_file_name, Length: 1341, dtype: object

In [45]:
df['subdir_video'] = df['subfolder'] + '_' + df['video_file_name']
df['subdir_video'] = df['subdir_video'].str.lower()

df['subdir_video_0'] = df.subdir_video.str.split('_').str[:5] + pd.Series('0' + df.subdir_video.str.split('_').str[5]).apply(lambda x: [x])
df['subdir_video_0'] = df['subdir_video_0'].str.join('_') 

# Only videos with 1 ape
df = df[df.max_number_chimps_per_video==1]
len(df)

129

In [46]:
# What we have locally
local_filenames = glob('/home/dl18206/Desktop/phd/data/panaf/chimp_videos/tanzania/**/*.mp4', recursive=True)
local_filenames = [x.split('/')[-2:] for x in local_filenames]
local_filenames = [x[-1].split('.')[0].lower() for x in local_filenames]

In [47]:
df[df.subdir_video.isin(local_filenames)].behavioral_context.value_counts().sum()
df[df.subdir_video_0.isin(local_filenames)].behavioral_context.value_counts().sum()
print(f"Total: {df[df.subdir_video.isin(local_filenames)].behavioral_context.value_counts().sum() + df[df.subdir_video_0.isin(local_filenames)].behavioral_context.value_counts().sum()}")

Total: 120


In [48]:
df[~df.subdir_video.isin(local_filenames)]

,new_row_id,country,research_site,genus,species,cam_coverage_area,location_metadata,habitat,utm_zone,utm_long,...,tool_use,vocalization,bipedal,camera_reaction,behavioral_context,other_species,additional_comments,record_type,subdir_video,subdir_video_0
230,231,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,19.91,trail,forest - colonising,36s,231829,...,no,no,no,no,travel,NaN,NaN,60s_video,uga_vid06_231829_9394396_20140426_ek000025,uga_vid06_231829_9394396_20140426_0ek000025
343,344,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,4.93,hill,savannah - wooded,36s,231921,...,no,no,no,no,travel,NaN,swelling is fairly tumescent but does not appe...,60s_video,uga_vid28_231921_9394324_20140115_ek000010,uga_vid28_231921_9394324_20140115_0ek000010
360,361,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,4.93,hill,savannah - wooded,36s,231921,...,no,no,no,yes,travel,NaN,seen at end of last video,60s_video,uga_vid28_231921_9394324_20140314_ek000012,uga_vid28_231921_9394324_20140314_0ek000012
581,582,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,5.39,clearing / hill,forest - colonising,36s,228947,...,no,no,no,no,feeding,NaN,"feeding on something and traveling, with a bri...",60s_video,uga_vid18_228947_9392134_20140629_ek000004,uga_vid18_228947_9392134_20140629_0ek000004
720,721,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,4.94,termite site,gallery forest,36s,229454,...,yes,no,no,no,tool use,NaN,barely visible at edge of frame; holding stick...,60s_video,uga_vid18_229454_9392080_20140907_pict0119,uga_vid18_229454_9392080_20140907_0pict0119
915,916,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,6.90,water source,gallery forest,36s,230395,...,no,no,no,no,unclear,NaN,appears to maybe drink water at stream,60s_video,uga_vid25_230395_9392009_20140221_pict0008,uga_vid25_230395_9392009_20140221_0pict0008
991,992,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,6.90,water source,gallery forest,36s,230395,...,no,no,no,no,resting,NaN,NaN,60s_video,uga_vid25_230395_9392009_20140821_pict0019,uga_vid25_230395_9392009_20140821_0pict0019
1049,1050,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,6.90,water source,gallery forest,36s,230395,...,no,no,no,no,travel,NaN,NaN,60s_video,uga_vid25_230395_9392009_20141028_pict0031,uga_vid25_230395_9392009_20141028_0pict0031
1157,1158,tanzania,ugalla_issa,Pan,troglodytes schweinfurthii,6.55,hill / rock,gallery forest,36s,230869,...,no,no,no,no,resting,NaN,NaN,60s_video,uga_viduppb_230869_9392585_20140903_pict0008,uga_viduppb_230869_9392585_20140903_0pict0008
